In [48]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

In [68]:
import os
filepath = '/Users/jindi/Dropbox-MIT/Clinical-Decision-Track/drugs/preprocessed/a-hydrocort.txt'
file = open(filepath)
texts = file.readlines()

In [102]:
grams = []
for text in texts:
    if len(text.split()) > 3:
        output = nlp.annotate(text, properties={
          'annotators': 'pos',
          'outputFormat': 'json'
          })
        for sentence in output['sentences']:
            tokens = sentence['tokens']
            index = 0
            while index < len(tokens):
                if ('NN' in tokens[index]['pos']) or ('VBG' in tokens[index]['pos']):
                    gram = tokens[index]['word'].lower()
                    offset = -1
                    while (index + offset >= 0) and (('JJ' in tokens[index+offset]['pos']) or 
                            ('VBG' in tokens[index+offset]['pos']) or ('VBN' in tokens[index+offset]['pos'])):
                        if ('included' in tokens[index+offset]['word'].lower()) or ('including' in tokens[index+offset]['word'].lower()):
                            break
                        gram = tokens[index+offset]['word'].lower() + ' ' + gram
                        offset -= 1
                    index += 1
                    while (index < len(tokens)) and (('NN' in tokens[index]['pos']) or 
                        ('IN' in tokens[index]['pos'] and (('JJ' in tokens[index+1]['pos']) or
                        ('DT' in tokens[index+1]['pos']) or ('NN' in tokens[index+1]['pos'])))):
                        if 'IN' in tokens[index]['pos']:
                            gram += ' ' + tokens[index]['word'].lower()
                            index += 1
                            while (index < len(tokens)) and (('JJ' in tokens[index]['pos']) 
                                                             or ('DT' in tokens[index]['pos'])):
                                gram += ' ' + tokens[index]['word'].lower()
                                index += 1
                        else:
                            gram += ' ' + tokens[index]['word'].lower()
                            index += 1
                    grams.append(gram)
                index += 1
    else:
        grams.append(text.rstrip('\n').lower())
print grams

[u'corticosteroid complications', u'dose', u'duration of therapy', u'adverse effects', u'lower pharmacologic dosages', u'adverse effects', u'duration of corticosteroid therapy', u'occurring during short-term therapy', u'weeks', u'occurring during long-term therapy', u'weeks', u'short-term effects', u'sodium', u'retention-related weight gain', u'fluid accumulation', u'hyperglycemia', u'glucose intolerance', u'hypokalemia', u'gastrointestinal upset', u'ulceration', u'reversible depression of the hypothalamus-pituitary-adrenal axis', u'mood changes', u'mild euphoria', u'insomnia', u'nervousness', u'restlessness', u'mania', u'catatonia', u'depression', u'delusions', u'hallucinations', u'violent behavior', u'long-term effects', u'hypothalamus-pituitary-adrenal activity suppression', u'cushingoid appearance', u'hirsutism', u'virilism', u'impotence', u'menstrual irregularities', u'peptic ulcer disease', u'cataracts', u'intraocular pressure/glaucoma', u'myopathy', u'osteoporosis', u'vertebral 